In [1]:
import os, glob, pandas as pd
import soundfile as sf

In [2]:
def getTime(ID):
    base = '../Data/asr_files/wav/*'
    for file in glob.glob(base):
        if os.path.basename(file).split('.')[0] == ID:
            f = sf.SoundFile(file)
            return (len(f) / f.samplerate)

In [3]:
def setUtterance(TG, ID, UTT):
    lineList = list()
    utterance = False
    xmax = False
    maxTime = getTime(ID)
    phones = False
    phoneList = list()
    wait = None
    counter = 0
    
    item2 = False
    item2List = list()
    item3 = False
    item3List = list()
    
    for line in TG:
        
        if 'intervals: size = 1' in line and phones == False:
            phones = True
            phoneList.append(line)
            pass
            
        if 'item [2]:' in line:
            phones = None
            LAB = open('../Data/LAB/{0}.lab'.format(ID), 'r', encoding='utf-8')
            labList = LAB.read().splitlines()
            num_lines = len(labList)
            
            lineList.append(phoneList[0].replace('1', str(num_lines-1)))
            
            xmin = '0'
            for i, item in enumerate(labList[1:]):
                lineList.append(phoneList[1].replace('1', str(i+1)))
                lineList.append(phoneList[2].replace('0', xmin))
                xmin = item.split()[0]
                lineList.append(phoneList[3].replace('0', xmin))
                lineList.append(phoneList[4].replace('\"\"', '"' + item.split()[2] + '"'))
            
        if phones and line not in phoneList:
            phoneList.append(line)
        
        if 'Utterance' in line:
            utterance = True
        if utterance and 'text' in line:
            lineList.append(line.replace('""', '"' + UTT + '"'))
                
        elif 'xmax' in line:
            lineList.append(line.replace('0', str(maxTime)))
            
        elif phones != True:
            """if phones == None:
                if 'item [2]' in line and item2 == False:
                    item2 = True
                    lineList.append(line)
                elif item2:
                    item2List.append(line)
                elif 'item [3]' in line and item3 == False:
                    item3 = True
                    item2 = False
                    lineList.append(line)
                elif item3:
                    item3List.append(line)"""
            lineList.append(line)
            
        else:
            pass
    base = '../Data/TextGrids/'
    out = base + ID + '.TextGrid'     
    
    newLL = list()
    for line in lineList:
        if 'xmax' in line:
            counter += 1
            if counter != 3:
                newLL.append(line)
        else:
            newLL.append(line)
        
    with open(out, 'w', encoding = 'utf-8') as f:
        for line in newLL:
            f.write("%s" % line)
            
    for x in item2List:
        print(x)

In [4]:
counter = 0
for file in glob.glob('../Data/asr_files/txt.done.data.rfs'):
    GOOD = open(file, 'r', encoding = 'utf-8')
    for line in GOOD:
        y = line.split(' ')
        ID = y[1]
        z = line.split('"')
        UTT = z[1]
        
        for file in glob.glob('../Data/TextGridTemplate.TextGrid'):
            TG = open(file, 'r', encoding = 'utf-8')
            
            setUtterance(TG, ID, UTT)
        #break
        counter += 1